In [8]:
from bitstring import BitArray, BitStream
from h264_stream import H264BitStream

In [9]:
input_file = open("samples/x264_test.264", "rb")
bitstream = BitArray(input_file)

start_code = '0x000001'


To find the starting position and the ending postion for each NALU interatively.

In [10]:
pos = 0
start_code_positions = []

while True:
    try:
        pos = bitstream.find(start_code, start=pos, bytealigned=True)[0]
        start_code_positions.append(pos)
        pos += 1
    except IndexError:
        break

start_code_positions.append(len(bitstream)) # add the last position


In [11]:
n = len(start_code_positions)

for i in range(n-1):
    print(f'Found {i+1} NAL at offset [{start_code_positions[i]}, {start_code_positions[i+1]}), size {start_code_positions[i+1] - start_code_positions[i]}')
    print(bitstream[start_code_positions[i]:start_code_positions[i+1]])

Found 1 NAL at offset [8, 240), size 232
0x0000016764001eacd940a03db011000003000100000300328f162d9600
Found 2 NAL at offset [240, 312), size 72
0x00000168ebe3cb22c0
Found 3 NAL at offset [312, 5824), size 5512
0x0000010605ffffaadc45e9bde6d948b7962cd820d923eeef78323634202d20636f7265203134382072323634332035633635373034202d20482e3236342f4d5045472d342041564320636f646563202d20436f70796c65667420323030332d32303135202d20687474703a2f2f7777772e766964656f6c616e2e6f7267...
Found 4 NAL at offset [5824, 6680), size 856
0x000001658884003bfffef74ebf029b54c22a0392570af6caa42659b9526b01f2a600000300000300000300908471ee6b95c93d668000002f6008d83261c814411710811c19e258428748ca0000030000030000030000030000030000030000030000030000030000978100
Found 5 NAL at offset [6680, 6864), size 184
0x000001419a246c437ffea7840000030000030000594000
Found 6 NAL at offset [6864, 7024), size 160
0x000001419e427885ff0000030000030000352100
Found 7 NAL at offset [7024, 7184), size 160
0x000001019e617442bf000003000003000047c000
Fo

Alternatively, the list of NAL units can be splited using BitArray.find()

In [18]:
nalus_ba = list(BitArray(input_file).split('0x000001', bytealigned=True))[1:]
# print(len(nalus_ba))
for nalu_ba in nalus_ba:
    print(nalu_ba)
    # nal to rbsp
    # enulation_prevention_three_byte
    nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
    nalu_bs = BitStream(nalu_ba)
    nb = H264BitStream(nalu_bs)
    params = {"forbidden_zero_bit" : nb.f(1),
              "nal_ref_idc" : nb.u(2),
              "nal_unit_type" : nb.u(5)}
    print(params)


0x0000016764001eacd940a03db011000003000100000300328f162d9600
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 7}
0x00000168ebe3cb22c0
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 8}
0x0000010605ffffaadc45e9bde6d948b7962cd820d923eeef78323634202d20636f7265203134382072323634332035633635373034202d20482e3236342f4d5045472d342041564320636f646563202d20436f70796c65667420323030332d32303135202d20687474703a2f2f7777772e766964656f6c616e2e6f7267...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 0, 'nal_unit_type': 6}
0x000001658884003bfffef74ebf029b54c22a0392570af6caa42659b9526b01f2a600000300000300000300908471ee6b95c93d668000002f6008d83261c814411710811c19e258428748ca0000030000030000030000030000030000030000030000030000030000978100
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 5}
0x000001419a246c437ffea7840000030000030000594000
{'forbidden_zero_bit': 0, 'nal_ref_idc': 2, 'nal_unit_type': 1}
0x000001419e427885ff0000030000030000352100
{'forbidden_zero_bit': 0, 'nal_re